In [6]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
from scipy.special import softmax

from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score
)

MODEL_NAME = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"
MODEL_PATH = f"../models/{MODEL_NAME}"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_PATH,
    local_files_only=True
)

config = AutoConfig.from_pretrained(MODEL_PATH)


In [7]:
def preprocess(text):
    new_text = []
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        # t = '' if t.startswith(('@', 'http')) and len(t) > 1 else t
        new_text.append(t)
    return " ".join(new_text)

def infer(text):
    labels = ['negative', 'neutral', 'positive']
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    output = ranking[0]
    label = labels[output]
    return text, label

In [8]:
def evaluate(predictions, true_labels):
    cls_report = classification_report(
        true_labels, 
        predictions
    )
    cnf_matrix = confusion_matrix(
        true_labels, 
        predictions
    )
    accuracy = accuracy_score(true_labels, predictions)

    print("Classification Report:\n", cls_report)
    print("Confusion Matrix:\n", cnf_matrix)
    print("Accuracy: ", accuracy)

def test_batch(csv_path):
    df = pd.read_csv(csv_path)
    data = df['text'].tolist()
    true_labels = df['expected_sentiment'].tolist()
    predictions = []

    for i in tqdm(range(len(data)), desc="Processing"):
        text = data[i]
        true_label = true_labels[i]
        processed_text, predicted_label = infer(text)
        predictions.append(predicted_label)

    evaluate(predictions, true_labels)


In [9]:
csv_path = '../data/sentiment_test_cases.csv'
test_batch(csv_path)

Processing:   0%|          | 0/498 [00:00<?, ?it/s]

Processing: 100%|██████████| 498/498 [00:38<00:00, 12.89it/s]

Classification Report:
               precision    recall  f1-score   support

    negative       0.87      0.86      0.87       177
     neutral       0.80      0.79      0.79       139
    positive       0.84      0.85      0.84       182

    accuracy                           0.84       498
   macro avg       0.84      0.84      0.84       498
weighted avg       0.84      0.84      0.84       498

Confusion Matrix:
 [[153   9  15]
 [ 14 110  15]
 [  8  19 155]]
Accuracy:  0.8393574297188755
